# TensorFlow2 Keras 推理


```{topic} 主题
TVM 对 TensorFlow1 模型的支持并不完善，为了提高开发效率，本节讨论如何将 TensorFlow1 模型转换为 TensorFlow2 的 Keras 模型，最终再将其转换为 ONNX 或者 TFLite 模型。
```

参考：[migrating_checkpoints](https://www.tensorflow.org/guide/migrate/migrating_checkpoints)

下面以模型 [resnet_v2_50](http://download.tensorflow.org/models/resnet_v2_50_2017_04_14.tar.gz) 为例展示。

需要克隆项目 [models](https://github.com/tensorflow/models)，然后执行如下操作。

In [1]:
import tensorflow as tf
try:
    tf1 = tf.compat.v1
except (ImportError, AttributeError):
    tf1 = tf
tf.get_logger().setLevel('ERROR')

2023-06-17 13:33:32.794252: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-17 13:33:35.626594: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-17 13:33:35.644448: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-17 13:33:55.703030: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


切换到 `models/research/slim` 目录下：

In [2]:
%cd /media/pc/data/lxw/ai/tasks/models/research/slim

/media/pc/data/lxw/ai/tasks/models/research/slim


将 TF1 升级为 TF2：

In [3]:
from nets import resnet_v2
import tf_slim as slim


class ResnetV2_50_block(tf.keras.layers.Layer):
    def __init__(self, trainable=False, 
                 name="resnet_v2_50", 
                 dtype=None, dynamic=False, **kwargs):
        super().__init__(trainable, name, dtype, dynamic, **kwargs)

    @tf1.keras.utils.track_tf1_style_variables
    def call(self, inputs):
        with slim.arg_scope(resnet_v2.resnet_arg_scope()):
            logits, end_points = resnet_v2.resnet_v2_50(
                inputs, 
                num_classes=1001,
                global_pool=True,
                is_training=self.trainable,
                scope=self.name
            )
        del end_points
        return tf.nn.softmax(logits)

class ResnetV2_50(tf.keras.Model):
    def __init__(self, trainable=False, 
                 name="resnet_v2_50", 
                 dtype=None, dynamic=False, **kwargs):
        super().__init__()
        self.block = ResnetV2_50_block(
            trainable=trainable, 
            name=name, dtype=dtype, 
            dynamic=dynamic, **kwargs
        )

    @tf.function(input_signature=[tf.TensorSpec([None, 224, 224, 3], 
                                                 tf.float32, name="data")])
    def call(self, inputs):
        x = self.block(inputs)
        return x

预处理：

In [4]:
from PIL import Image
import numpy as np
from nets import resnet_v2
from preprocessing.preprocessing_factory import get_preprocessing
import tf_slim as slim

preprocessing = get_preprocessing("resnet_v2_50")
image_size = 224
path = '/media/pc/data/board/arria10/lxw/data/test/cat.png' # 将要预测的图片路径
preprocessing = get_preprocessing("resnet_v2_50")


@tf.function
def preprocess_image(image, output_height, output_width):
    # image = tf.constant(image)
    processed_image = preprocessing(image, output_height, output_width)
    return processed_image/256
with Image.open(path) as im:
    if im.mode != "RGB":
        im.convert("RGB")
    image = np.asarray(im)
np_processed_image = preprocess_image(image, image_size, image_size)
np_processed_images = np.expand_dims(np_processed_image, axis=0)

2023-06-17 13:34:44.643279: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


前向推理：

In [5]:
model = ResnetV2_50()
model(tf.ones(shape=(1, 224, 224, 3), dtype=tf.float32))
ckpt = tf.train.Checkpoint(model=model)
ckpt.restore("/tmp/checkpoints/resnet_v2_50.ckpt") # 更新模型参数
outputs = model(np_processed_images)
outputs = outputs.numpy()

/media/pc/data/tmp/cache/conda/envs/tvmz/lib/python3.10/site-packages/tensorflow/python/keras/engine/base_layer.py:2212: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/media/pc/data/tmp/cache/conda/envs/tvmz/lib/python3.10/site-packages/tensorflow/python/keras/engine/base_layer.py:1345: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '
/media/pc/data/tmp/cache/conda/envs/tvmz/lib/python3.10/site-packages/keras/legacy_tf_layers/base.py:627: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  self.updates, tf.compat.v1.GraphKeys.UPDATE_OPS


NotFoundError: Error when restoring from checkpoint or SavedModel at /tmp/checkpoints/resnet_v2_50.ckpt: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for /tmp/checkpoints/resnet_v2_50.ckpt
Please double-check that the path is correct. You may be missing the checkpoint suffix (e.g. the '-1' in 'path/to/ckpt-1').

打印标签信息：

In [ ]:
from github import Github

g = Github(user_agent="xinetzone")
repo = g.get_repo("tensorflow/models")
label_content = repo.get_contents("research/slim/datasets/imagenet_lsvrc_2015_synsets.txt")
imagenet_labels = label_content.decoded_content.decode().split()
assert len(imagenet_labels) == 1000
metadata = repo.get_contents("research/slim/datasets/imagenet_metadata.txt")
imagenet_metadata = metadata.decoded_content.decode().splitlines()
synset_to_human = {}
for metadata in imagenet_metadata:
    name, value = metadata.split("\t")
    synset_to_human[name] = value
name2id = {name: k+1 for k, name in enumerate(imagenet_labels)}

topk = 5
sorted_inds = outputs[0].argsort()[::-1]
for sorted_ind in sorted_inds[:topk]:
    label = synset_to_human[imagenet_labels[sorted_ind-1]]
    print(f"{sorted_ind-1}: {label.ljust(20)}\t{outputs[0][sorted_ind-1]}")

In [ ]:
model.summary()

将其模型和参数与加载下来：

In [ ]:
# # model = ResnetV2_50()
# inputs = tf.keras.Input(shape=(224, 224, 3), dtype=tf.float32, name="data")
# outputs = model(inputs)
# model2 = tf.keras.Model(inputs=inputs, outputs=outputs, name="resnet_v2_50_model")

# model2.save(module_with_signature_path)

In [ ]:
module_with_signature_path = "temp/resnet_v2_50_keras"
model.save(module_with_signature_path)
imported_with_signatures = tf.saved_model.load(module_with_signature_path)
infer = imported_with_signatures.signatures['serving_default']
labeling = infer(tf.constant(np_processed_images))

## 转换为 ONNX 模型

[Keras 模型转换 ONNX](https://onnxruntime.ai/docs/tutorials/tf-get-started.html)：

In [ ]:
import tf2onnx
import onnx

input_signature = [tf.TensorSpec([1, 224, 224, 3], tf.float32, name="data")]
# Use from_function for tf functions
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature, opset=13)
onnx.save(onnx_model, "temp/resnet_v2_50.onnx")

构建库：

In [ ]:
import set_env
from tvm.relay.frontend import from_onnx

shape_dict = {"data": [1, 224, 224, 3]}

graph_def = onnx.load("temp/resnet_v2_50.onnx")
mod, params = from_onnx(
    graph_def,
    shape_dict,
    opset=13,
    freeze_params=True
)

推理：

In [ ]:
import tvm
from tvm import relay

with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, "llvm", params=params)
    
inputs_dict = {"data": np_processed_images}
mlib_proxy = tvm.contrib.graph_executor.GraphModule(lib["default"](tvm.cpu()))
mlib_proxy.run(**inputs_dict)

验证一致性：

In [ ]:
np.testing.assert_allclose(
    labeling['output_1'].numpy(), 
    mlib_proxy.get_output(0).numpy(),
    rtol=1e-07, atol=1e-5
)

## 转换为 TFLite 模型

In [ ]:
import tensorflow as tf

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(module_with_signature_path)
tflite_model = converter.convert()

# Save the model.
with open('temp/resnet_v2_50.tflite', 'wb') as f:
    f.write(tflite_model)

加载 TFLite 模型：

In [ ]:
import tflite


with open('temp/resnet_v2_50.tflite', "rb") as fp:
    tflite_model_buf = fp.read()

tflite_model = tflite.Model.GetRootAsModel(tflite_model_buf, 0)
mod, params = relay.frontend.from_tflite(
    tflite_model, shape_dict=shape_dict, 
    dtype_dict={"data": "float32"}
)
desired_layouts = {
    # 'image.resize2d': ['NCHW'],
    'nn.conv2d': ['NCHW', 'default'],
    'nn.max_pool2d': ['NCHW', 'default'],
    'nn.avg_pool2d': ['NCHW', 'default'],
}
# NHWC 将布局转换为 NCHW 且移除未使用算子
seq = tvm.transform.Sequential([
    relay.transform.RemoveUnusedFunctions(),
    relay.transform.ConvertLayout(desired_layouts)
])
with tvm.transform.PassContext(opt_level=3):
    mod = seq(mod)

验证结果一致性：

In [ ]:
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, "llvm", params=params)
    
inputs_dict = {"data": np_processed_images}
mlib_proxy = tvm.contrib.graph_executor.GraphModule(lib["default"](tvm.cpu()))
mlib_proxy.run(**inputs_dict)
np.testing.assert_allclose(
    labeling['output_1'].numpy(), 
    mlib_proxy.get_output(0).numpy(),
    rtol=1e-07, atol=1e-5
)

```{warning}
TFLite 转换出现了问题，暂时搁置。
```